In [2]:
import pandas as pd
data=pd.read_csv('data.csv')


In [3]:
print(data.shape)

(900000, 46)


In [6]:
print(data.columns)#查看有多少列信息，IS_LOST是lable，

Index(['Unnamed: 0', 'MONTH_ID', 'USER_ID', 'INNET_MONTH', 'ACCT_FEE',
       'CALL_DURA', 'NO_ROAM_LOCAL_CALL_DURA', 'NO_ROAM_GN_LONG_CALL_DURA',
       'GN_ROAM_CALL_DURA', 'CDR_NUM', 'NO_ROAM_CDR_NUM',
       'NO_ROAM_LOCAL_CDR_NUM', 'NO_ROAM_GN_LONG_CDR_NUM', 'GN_ROAM_CDR_NUM',
       'P2P_SMS_CNT_UP', 'TOTAL_FLUX', 'LOCAL_FLUX', 'GN_ROAM_FLUX',
       'CALL_DAYS', 'CALLING_DAYS', 'CALLED_DAYS', 'CALL_RING', 'CALLING_RING',
       'CALLED_RING', 'CERT_AGE', 'IS_LOST', 'CUST_SEX_1', 'CUST_SEX_2',
       'CUST_SEX_3', 'TERM_TYPE_2', 'TERM_TYPE_3', 'TERM_TYPE_4',
       'CONSTELLATION_DESC_1', 'OS_DESC_IOS', 'CREDIT_LEVEL_0',
       'CREDIT_LEVEL_65', 'CREDIT_LEVEL_66', 'CREDIT_LEVEL_67', 'VIP_LVL_0',
       'VIP_LVL_2', 'VIP_LVL_3', 'VIP_LVL_4', 'VIP_LVL_99', 'AGREE_EXP_DATE_S',
       'AGREE_EXP_DATE_M', 'AGREE_EXP_DATE_L'],
      dtype='object')


In [32]:
print(data[['CUST_SEX_1','CUST_SEX_2','CUST_SEX_3','IS_LOST']].head())
#查看性别信息，可以看到CUST_SEX_1，CUST_SEX_2是男或女(两行信息是互斥的)，CUST_SEX_3是nan信息即性别信息是否缺失(与前两个字段数据互斥)

   CUST_SEX_1  CUST_SEX_2  CUST_SEX_3  IS_LOST
0           1           0           0      NaN
1           1           0           0      NaN
2           1           0           0      NaN
3           0           1           0      NaN
4           0           1           0      NaN


In [28]:
print(data['IS_LOST'].count())#count 函数是统计的非空的数量，看出来大部分是空的，差不多只有1/3

300309


In [29]:
print(data["IS_LOST"].value_counts())#看出来正负样本的分布情况。负样本差不多是正样本的30倍

0.0    290596
1.0      9713
Name: IS_LOST, dtype: int64


In [38]:
print((data["CUST_SEX_1"]*data["IS_LOST"]).value_counts())
print((data["CUST_SEX_2"]*data["IS_LOST"]).value_counts())
print((data["CUST_SEX_3"]*data["IS_LOST"]).value_counts())
#我们来查看不同的性别信息对是否流失统计情况

0.0    293699
1.0      6610
dtype: int64
0.0    297441
1.0      2868
dtype: int64
0.0    300074
1.0       235
dtype: int64


In [55]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import numpy as np
#获得有用的信息,标签不为空的信息
avil_data=data[data.IS_LOST.isnull().values==False]
print(avil_data.shape)
#总体数据量太大，我随机抽取3w行来分析，也看出来数据的正负赝=样本分布也没有很大改变
sample_data=avil_data.sample(10000)
print(sample_data.shape)
print(sample_data.IS_LOST.value_counts())
#做多维度方差分析
formula = 'IS_LOST~INNET_MONTH+ACCT_FEE+NO_ROAM_LOCAL_CALL_DURA+NO_ROAM_GN_LONG_CALL_DURA+\
            GN_ROAM_CALL_DURA+CDR_NUM+NO_ROAM_CDR_NUM+NO_ROAM_LOCAL_CDR_NUM+NO_ROAM_GN_LONG_CDR_NUM+\
            GN_ROAM_CDR_NUM+P2P_SMS_CNT_UP+TOTAL_FLUX+LOCAL_FLUX+GN_ROAM_FLUX+CALL_DAYS+CALLING_DAYS+CALLED_DAYS+\
            CALL_RING+CALLING_RING+CALLED_RING+CERT_AGE'
anova_results = anova_lm(ols(formula,sample_data).fit())
print (anova_results)
#以下是数据分析的结果是不是F值越大越好，我也不太熟悉，之后查查资料看看

(300309, 46)
(10000, 46)
0.0    9708
1.0     292
Name: IS_LOST, dtype: int64
                               df      sum_sq    mean_sq           F  \
INNET_MONTH                   1.0    3.882888   3.882888  148.768864   
ACCT_FEE                      1.0    1.090190   1.090190   41.769514   
NO_ROAM_LOCAL_CALL_DURA       1.0    0.940622   0.940622   36.038980   
NO_ROAM_GN_LONG_CALL_DURA     1.0    0.207392   0.207392    7.946029   
GN_ROAM_CALL_DURA             1.0    0.206459   0.206459    7.910281   
CDR_NUM                       1.0    0.100789   0.100789    3.861632   
NO_ROAM_CDR_NUM               1.0    0.074463   0.074463    2.852968   
NO_ROAM_LOCAL_CDR_NUM         1.0    0.003337   0.003337    0.127856   
NO_ROAM_GN_LONG_CDR_NUM       1.0    0.097479   0.097479    3.734810   
GN_ROAM_CDR_NUM               1.0    0.009749   0.009749    0.373521   
P2P_SMS_CNT_UP                1.0    0.004138   0.004138    0.158527   
TOTAL_FLUX                    1.0    0.585525   0.585525   

/home/lab713/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/lab713/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/lab713/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [122]:
#我们选贡献比较大的几个字段数据生成训练样本
from sklearn.utils import shuffle
features=['INNET_MONTH','ACCT_FEE','NO_ROAM_LOCAL_CALL_DURA','NO_ROAM_GN_LONG_CALL_DURA',\
                     'CDR_NUM','NO_ROAM_CDR_NUM','TOTAL_FLUX','CALL_DAYS','CALLED_DAYS',\
                    'CERT_AGE','CUST_SEX_1','CUST_SEX_2','CUST_SEX_3','IS_LOST']
train_data=avil_data[features]
#打乱数据
train_data=shuffle(train_data)
print(train_data.shape)
X_data,y_data=train_data[features[0:13]].astype('int64'),train_data.IS_LOST.apply(lambda x: int(x))
print(X_data.shape)
print(y_data.shape)


(300309, 14)
(300309, 13)
(300309,)


In [123]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_data,y_data)
print(X_train.shape)
print(X_test.shape)
print(y_data.head())

(225231, 13)
(75078, 13)
800246    0
843470    0
802500    0
730770    0
641226    0
Name: IS_LOST, dtype: int64


In [126]:
from sklearn.metrics import classification_report
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_predict=clf.predict(X_test)
print(classification_report(y_predict,y_test))
#可以看到对正样本的预测不是很准确

             precision    recall  f1-score   support

          0       0.97      0.98      0.97     72173
          1       0.31      0.25      0.28      2905

avg / total       0.94      0.95      0.95     75078



In [128]:
from sklearn.externals.six import StringIO
import pydotplus

feature_name = features[:-1]
target_name = ['0','1']

#把决策数可视化，输出的图片会有点大，打开图片加载时比较慢
dot_data = StringIO()
tree.export_graphviz(clf,out_file = dot_data,feature_names=feature_name,
                     class_names=target_name,filled=True,rounded=True,
                     special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("DecTree.pdf")
print('Visible tree plot saved as pdf.')

Visible tree plot saved as pdf.
